In [1]:
!pip install pandas regex

In [2]:
import os
cwd = os.getcwd()
files = os.listdir(cwd)
print('files in %r: %s' % (cwd,files))

files in '/Users/gerrylorinanto': ['.config', 'Music', 'Flask_and_SwaggerUI.ipynb', 'Gelo', '.condarc', 'Chalengge 231103.ipynb', 'Untitled5.ipynb', 'Untitled1.ipynb', 'Challenge 231103.ipynb', '.DS_Store', 'KK Chalengge .ipynb', '.CFUserTextEncoding', '.xonshrc', 'nama_database.db', 'Untitled3.ipynb', 'Untitled.ipynb', '.zshrc', 'File data Import ', 'Untitled4.ipynb', 'Pictures', 'emoji-2.8.0-py2.py3-none-any.whl', '.zprofile', 'Mini challenge 231113.ipynb', '.zsh_history', 'Untitled2.ipynb', '.ipython', 'Desktop', 'Library', '.matplotlib', 'KK Chalengge - Final.ipynb', '.cups', 'flagged', 'Public', '.idlerc', '.tcshrc', '.anaconda', 'Movies', 'Applications', 'app.py', 'KK Gold Challenge Analisa Statistik.ipynb', 'log', '.Trash', '.ipynb_checkpoints', '.jupyter', 'Documents', '.mysql_history', '.bash_profile', 'KK Flask_Swagger Gerr.ipynb', 'anaconda3', 'Downloads', '.continuum', '.zsh_sessions', '.conda', 'abusive_database.db']


In [3]:
%cd '/Users/gerrylorinanto/Gelo/Gerry/Binar/Text Data Gold Challenge'

/Users/gerrylorinanto/Gelo/Gerry/Binar/Text Data Gold Challenge


In [ ]:
#sebelum di msukkan database
import re
import pandas as pd
import demoji

from flask import Flask, jsonify

app = Flask(__name__)

from flask import request
from flasgger import Swagger, LazyJSONEncoder, swag_from

app.json_encoder = LazyJSONEncoder

swagger_template = {
    "info": {
        "title": "API Documentation for Data Processing Gold Challenge",
        "version": "1.0.0",
        "description": "Dokumentasi API untuk Data Processing dan Modeling",
    },
    "host": "127.0.0.1:5000",
}
swagger_config = {
    "headers": [],
    "specs": [
        {"endpoint": "docs", "route": "/docs.json",}
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/",
}
swagger = Swagger(app, template=swagger_template, config=swagger_config)

# mengambil dafatar kata abusive dan kamus alay dari file csv dan disimpan dalam bentuk dictionary.
abusive_words_df = pd.read_csv("abusive.csv", encoding="ascii")
abusive_words_list = abusive_words_df["ABUSIVE"].tolist()

alay_words_df = pd.read_csv("new_kamusalay.csv", encoding="ISO-8859-1")
alay_words_dict = dict(zip(alay_words_df['anakjakartaasikasik'], alay_words_df['anak jakarta asyik asyik']))

@swag_from(
    "/Users/gerrylorinanto/Gelo/Gerry/Binar/Supporting Material/Chapter 3/Script-for-Channel-3-Topic-3-on-Binar-Academy-Data-Science-Bootcamp-master/docs/hello_world.yml",
    methods=["GET"],
)
@app.route("/", methods=["GET"])
def hello_world():
    json_response = {
        "status_code": 200,
        "description": "Menyapa Hello World",
        "data": "Hello World",
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from(
    "/Users/gerrylorinanto/Gelo/Gerry/Binar/Text Data Gold Challenge/Clean_text.yml",
    methods=["POST"],
)
@app.route("/clean_text", methods=["POST"])
def clean_text():
    text = request.form["text"]
    cleaned_text = demoji.replace(text, " ")
    cleaned_text = re.sub(r"[^\w\s]", " ", cleaned_text)
    json_response = {
        "status_code": 200,
        "description": "Text cleaned successfully",
        "data": cleaned_text,
    }
    return jsonify(json_response)

@swag_from(
    "/Users/gerrylorinanto/Gelo/Gerry/Binar/Text Data Gold Challenge/Upload_to_CSV_EP.yml",
    methods=["POST"],
)
@app.route("/upload_CSV_EP", methods=["POST"])
def upload_csv():
    file = request.files.getlist("file")[0]
    df = pd.read_csv(file, encoding="ISO-8859-1")

    df["cleaned_text"] = df.iloc[:, 0].apply(clean_text_with_custom_rules)
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: re.compile(r'[\\x]+[a-z0-9]{2}').sub('', x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: ' '.join(alay_words_dict.get(word.lower(), word) for word in x.split()))
    df["cleaned_text"] = df["cleaned_text"].apply(remove_abusive_words)

    json_response = {
        "status_code": 200,
        "description": "CSV cleaned successfully",
        "data": df["cleaned_text"].tolist(),
    }
    response_data = jsonify(json_response)
    return response_data


def clean_text_with_custom_rules(text):
    """
    Cleans the text by removing non-alphanumeric characters and multiple spaces.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: The cleaned text.
    """
    # Remove non-alphanumeric characters
    cleaned_text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text


def remove_abusive_words(text):
    """
    Removes abusive words from the text based on the abusive_words_list.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: The text with removed abusive words.
    """
    for word in abusive_words_list:
        # Menambahkan spasi di antara kata yang di-remove
        text = text.replace(word, '')
    return text

if __name__ == "__main__":
    app.run()

In [4]:
#setelah dimasukkan database
import re
import sqlite3
import pandas as pd
import demoji

from flask import Flask, jsonify, request
from flasgger import Swagger, LazyJSONEncoder, swag_from

app = Flask(__name__)
app.json_encoder = LazyJSONEncoder

swagger_template = {
    "info": {
        "title": "API Documentation for Data Processing Gold Challenge",
        "version": "1.0.0",
        "description": "Dokumentasi API untuk Data Processing dan Modeling",
    },
    "host": "127.0.0.1:5000",
}
swagger_config = {
    "headers": [],
    "specs": [
        {"endpoint": "docs", "route": "/docs.json",}
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/",
}
swagger = Swagger(app, template=swagger_template, config=swagger_config)

# Inisialisasi Database
def initialize_database():
    conn = sqlite3.connect('goldchallenge_database.db')
    cursor = conn.cursor()

    # Buat tabel jika belum ada
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS cleaned_data (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            original_text TEXT,
            cleaned_text TEXT
        )
    ''')

    conn.commit()
    conn.close()

# Panggil fungsi inisialisasi pada saat aplikasi dijalankan
initialize_database()

# Load daftar kata abusive dari abusive.csv
abusive_words_df = pd.read_csv("abusive.csv", encoding="ascii")
abusive_words_list = abusive_words_df["ABUSIVE"].tolist()

# Load kamus kata alay dari kamusalay.csv
alay_words_df = pd.read_csv("new_kamusalay.csv", encoding="ISO-8859-1")
alay_words_dict = dict(zip(alay_words_df['anakjakartaasikasik'], alay_words_df['anak jakarta asyik asyik']))

@swag_from(
    "/Users/gerrylorinanto/Gelo/Gerry/Binar/Supporting Material/Chapter 3/Script-for-Channel-3-Topic-3-on-Binar-Academy-Data-Science-Bootcamp-master/docs/hello_world.yml",
    methods=["GET"],
)
@app.route("/", methods=["GET"])
def hello_world():
    json_response = {
        "status_code": 200,
        "description": "Menyapa Hello World",
        "data": "Hello World",
    }

    response_data = jsonify(json_response)
    return response_data

@swag_from(
    "/Users/gerrylorinanto/Gelo/Gerry/Binar/Text Data Gold Challenge/Clean_text.yml",
    methods=["POST"],
)
@app.route("/clean_text", methods=["POST"])
def clean_text():
    text = request.form["text"]
    cleaned_text = demoji.replace(text, " ")
    cleaned_text = re.sub(r"[^\w\s]", " ", cleaned_text)
    json_response = {
        "status_code": 200,
        "description": "Text cleaned successfully",
        "data": cleaned_text,
    }
    return jsonify(json_response)

@swag_from(
    "/Users/gerrylorinanto/Gelo/Gerry/Binar/Text Data Gold Challenge/Upload_to_CSV_EP.yml",
    methods=["POST"],
)
@app.route("/upload_CSV_EP", methods=["POST"])
def upload_csv():
    file = request.files.getlist("file")[0]
    df = pd.read_csv(file, encoding="ISO-8859-1")

    # Proses pembersihan
    df["cleaned_text"] = df.iloc[:, 0].apply(clean_text_with_custom_rules)
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: re.compile(r'[\\x]+[a-z0-9]{2}').sub('', x))
    df["cleaned_text"] = df["cleaned_text"].apply(lambda x: ' '.join(alay_words_dict.get(word.lower(), word) for word in x.split()))
    df["cleaned_text"] = df["cleaned_text"].apply(remove_abusive_words)

    # Simpan data ke database
    conn = sqlite3.connect('goldchallenge_database.db')
    df['cleaned_text'].to_sql('cleaned_data', conn, if_exists='append', index=False)
    conn.close()

    json_response = {
        "status_code": 200,
        "description": "CSV cleaned and data saved successfully",
        "data": df["cleaned_text"].tolist(),
    }
    response_data = jsonify(json_response)
    return response_data

def clean_text_with_custom_rules(text):
    """
    Cleans the text by removing non-alphanumeric characters and multiple spaces.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: The cleaned text.
    """
    # Remove non-alphanumeric characters
    cleaned_text = re.sub(r'[^a-zA-Z0-9]', ' ', text)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)

    return cleaned_text

def remove_abusive_words(text):
    """
    Removes abusive words from the text based on the abusive_words_list.

    Parameters:
    - text (str): The input text.

    Returns:
    - str: The text with removed abusive words.
    """
    for word in abusive_words_list:
        # Menambahkan spasi di antara kata yang di-remove
        text = text.replace(word, '')
    return text

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [5]:
import os
import sqlite3

def check_database():
    if os.path.exists('goldchallenge_database.db'):
        conn = sqlite3.connect('goldchallenge_database.db')
        cursor = conn.cursor()

        # Check if the table exists
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='cleaned_data';")
        table_exists = cursor.fetchone() is not None

        conn.close()

        if table_exists:
            print("The 'cleaned_data' table exists in the database.")
        else:
            print("The 'cleaned_data' table does not exist in the database.")
    else:
        print("The database file 'goldchallenge_database.db' does not exist.")

if __name__ == "__main__":
    check_database()


The 'cleaned_data' table exists in the database.
